In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import time
import pandas as pd
from bs4 import BeautifulSoup
from itertools import repeat
from itertools import groupby

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import re
!pip install lxml

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless") # 로컬에서만 끌 수 있음
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")

In [ ]:
driver = webdriver.Chrome(options = chrome_options)
news_url = 'https://finance.yahoo.com/news/akamai-reports-third-quarter-2025-210100645.html'
driver.get(news_url)

In [9]:
### 본문, 표 모두 text 추출
def get_news_content(html_paths):
    try:
        driver = webdriver.Chrome(options = chrome_options)
    except Exception as e:
        print(f"WebDriver 실행 중 오류 발생 : {e}")
        return []

    wait = WebDriverWait(driver, 10)
    full_news = []

    for i, url in enumerate(html_paths, 1):
        article_content = ""
        try:        
            print(f"{i}번째 기사 처리 중")
            driver.get(url)

            try: # 있으면 클릭
                cookie_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@name='agree']")))
                cookie_btn.click()
                time.sleep(2) # 팝업 사라질 시간
            except TimeoutException: # 팝업 없으면
                pass

            # 제목
            title_path = '//*[@id="main-content-wrapper"]/div/article/div[2]/div[2]/h1'
            title = wait.until(EC.visibility_of_element_located((By.XPATH, title_path))).text

            # 에디터 
            editor_path = '//*[@id="main-content-wrapper"]/div/article/div[3]/div[1]/div/div[1]'
            editor = driver.find_element(By.XPATH, editor_path).text

            # 날짜
            date_path = '//*[@id="main-content-wrapper"]/div/article/div[3]/div[1]/div/div[2]/time'
            date = driver.find_element(By.XPATH, date_path).text
            # datetime = driver.find_element(By.XPATH, date_path).get_attribute("datetime")

            # 텍스트
            all_texts_path1 = '//*[@id="main-content-wrapper"]/div/article/div[4]/div[2]/div[1]'

            try:
                all_texts_elements1 = driver.find_elements(By.XPATH, all_texts_path1)
                print(len(all_texts_elements1))

                for element in all_texts_elements1:
                    article_content += element.text + "\n"

            except Exception as e:
                print(f"오류 발생 : {e}")

            # 더보기 누르기
            button_path = '//*[@id="main-content-wrapper"]/div/article/div[4]/div/div[2]/button'
            button = wait.until(EC.element_to_be_clickable((By.XPATH, button_path)))
            button.click()
            time.sleep(5)

            all_texts_path2 = '//*[@id="main-content-wrapper"]/div/article/div[4]/div[2]/div[3]'
            
            try:
                all_texts_elements2 = driver.find_elements(By.XPATH, all_texts_path2)

                for element in all_texts_elements2:
                    article_content += element.text + "\n"

            except Exception as e:
                print(f"오류 발생 : {e}")

            news = {'title' : title, 'editor' : editor, 'date' : date, 'text' : article_content, 'html' : url}
            full_news.append(news)

            print(f">>> {i}번째 기사 처리 완료 !")

        except Exception as e:
            print(f"오류 발생 : {e}")
            continue

    driver.quit()
    full_news = pd.DataFrame(full_news)
    return full_news


In [3]:
def _process_table(table_html):
    """테이블에서 NaN이 많은 열 제거 후 반환"""
    try:
        df = pd.read_html(table_html)[0]
        # NaN 열 제거
        no_nan_counts = df.count(axis = 0)
        df = df.loc[:, no_nan_counts > 1]
        return df

    except Exception as e:
        print(f">> 테이블 파싱 실패 : {e}")
        return None 

In [4]:
def _process_elements(elements, text_tags):
    """
    Selenium 요소(elements) 리스트를 받아 텍스트/테이블 분리하여 리스트로 반환
    -> 텍스트 + 테이블 리스트, 텍스트 only 문자열 반환
    """
    local_scraped_content = [] 
    local_article_content = ""

    for element in elements:
        tag = element.tag_name

        if tag in text_tags:
            text = element.text
            if text.strip():
                local_scraped_content.append(text)
                local_article_content += text + "\n"

        elif tag == "div":
            # 광고인지 확인
            class_attr = element.get_attribute("class") or ""
            testid_attr = element.get_attribute("data-testid")
            # 광고라면
            if 'yf-eondl' in class_attr or 'inarticle-ad' == testid_attr:
                print(f">> 광고 태그 발견 - 필터링함")
                pass
            # 광고가 아닌 실제 테이블인 경우
            else:
                try:
                    table_inside = element.find_element(By.XPATH, ".//table")
                    print(f">> 내부에 테이블 존재")
                    table_html = table_inside.get_attribute("outerHTML")
                    df = _process_table(table_html)

                    if df is not None:
                        local_scraped_content.append(df)
                    
                except NoSuchElementException:
                    print(f">> 내부에 테이블 존재하지 않음 - 광고/필터링")
        
        elif tag == "figure":
            pass

        elif tag == "table":
            try:
                table_html = element.get_attribute("outerHTML")
                df = _process_table(table_html)
                if df is not None:
                    local_scraped_content.append(df)
            except Exception as e:
                print(f">> 테이블 파싱 실패 : {e}")

        else:
            print(f">> 태그 분류 실패 - {tag}")
            continue
    
    return local_scraped_content, local_article_content

In [5]:
def get_news_content(html_paths):
    try:
        driver = webdriver.Chrome(options = chrome_options)
    except Exception as e:
        print(f"WebDriver 실행 중 오류 발생 : {e}")
        return []

    wait = WebDriverWait(driver, 10)
    scraped_news = []
    text_tags = ['p', 'ul', 'ol', 'h2', 'h3', 'h4', 'h5', 'h6', 'blockquote']
    # 그 외 div : 내부에 테이블 존재 or 광고 등 / figure : 이미지

    for i, url in enumerate(html_paths, 1):
        article_content = ""
        scraped_content = []
        
        try:        
            print(f"{i}번째 기사 처리 중")
            driver.get(url)

            # 쿠키 팝업 처리
            try: # 있으면 클릭
                cookie_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@name='agree']")))
                cookie_btn.click()
                time.sleep(2) # 팝업 사라질 시간
            except TimeoutException: # 팝업 없으면
                pass

            # 제목
            title_path = '//*[@id="main-content-wrapper"]/div/article/div[2]/div[2]/h1'
            title = wait.until(EC.visibility_of_element_located((By.XPATH, title_path))).text

            # 에디터 
            editor_path = '//*[@id="main-content-wrapper"]/div/article/div[3]/div[1]/div/div[1]'
            editor = driver.find_element(By.XPATH, editor_path).text

            # 날짜
            date_path = '//*[@id="main-content-wrapper"]/div/article/div[3]/div[1]/div/div[2]/time'
            date = driver.find_element(By.XPATH, date_path).text

            # 텍스트 1
            all_texts_path1 = '//*[@id="main-content-wrapper"]/div/article/div[4]/div/div[1]'
            all_texts_path1 = all_texts_path1 + "/*"

            try:
                all_texts_elements1 = driver.find_elements(By.XPATH, all_texts_path1)
                print(f"Part 1: {len(all_texts_elements1)}개 요소 발견")

                scraped, article = _process_elements(all_texts_elements1, text_tags)
                scraped_content.extend(scraped)
                article_content += article

            except Exception as e:
                print(f"Part 1 스크래핑 오류 : {e}")

            # 더보기 누르기
            button_path = '//*[@id="main-content-wrapper"]/div/article/div[4]/div/div[2]/button'
            button = driver.find_element(By.XPATH, button_path)
            driver.execute_script("arguments[0].click();", button)
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="main-content-wrapper"]/div/article/div[4]/div/div[3]')))

            # 텍스트 2
            all_texts_path2 = '//*[@id="main-content-wrapper"]/div/article/div[4]/div/div[3]'
            all_texts_path2 = all_texts_path2 + "/*"
            
            try:
                all_texts_elements2 = driver.find_elements(By.XPATH, all_texts_path2)
                print(f"Part 2: {len(all_texts_elements2)}개 요소 발견")

                scraped, article = _process_elements(all_texts_elements2, text_tags)
                scraped_content.extend(scraped)
                article_content += article

            except Exception as e:
                print(f"Part 2 스크래핑 오류 : {e}")

            # metadata, 내용 분리해서 저장
            metadata = {'title' : title, 'editor' : editor, 'date' : date, 'html' : url}
            scraped_news.append({'metadata' : metadata, 'content' : scraped_content})

            print(f">>> {i}번째 기사 처리 완료 !")

        except Exception as e:
            print(f"오류 발생 : {e}")
            continue

    driver.quit()

    return scraped_news

In [6]:
html_paths = ['https://finance.yahoo.com/news/akamai-reports-third-quarter-2025-210100645.html']
scraped_content = get_news_content(html_paths)

1번째 기사 처리 중
Part 1: 16개 요소 발견
>> 광고 태그 발견 - 필터링함
>> 광고 태그 발견 - 필터링함
Part 2: 73개 요소 발견
>> 광고 태그 발견 - 필터링함
>> 내부에 테이블 존재
>> 내부에 테이블 존재
>> 광고 태그 발견 - 필터링함
>> 내부에 테이블 존재
>> 내부에 테이블 존재
>> 내부에 테이블 존재
>> 내부에 테이블 존재
>> 내부에 테이블 존재
>> 내부에 테이블 존재
>> 내부에 테이블 존재
>> 내부에 테이블 존재
>> 내부에 테이블 존재
>> 내부에 테이블 존재
>> 내부에 테이블 존재
>> 내부에 테이블 존재
>> 내부에 테이블 존재
>> 내부에 테이블 존재
>> 광고 태그 발견 - 필터링함
>> 광고 태그 발견 - 필터링함
>> 광고 태그 발견 - 필터링함
>> 광고 태그 발견 - 필터링함
>> 광고 태그 발견 - 필터링함
>> 광고 태그 발견 - 필터링함
>> 광고 태그 발견 - 필터링함
>> 광고 태그 발견 - 필터링함
>> 광고 태그 발견 - 필터링함
>>> 1번째 기사 처리 완료 !


In [12]:
scraped_content

[{'metadata': {'title': 'AKAMAI REPORTS THIRD QUARTER 2025 FINANCIAL RESULTS',
   'editor': 'PR Newswire',
   'date': 'Fri, November 7, 2025 at 6:01 AM GMT+9',
   'html': 'https://finance.yahoo.com/news/akamai-reports-third-quarter-2025-210100645.html'},
  'content': ['Third  quarter revenue of $1.055 billion, up 5% year-over-year and up 4% when adjusted for foreign exchange*',
   'Cloud Infrastructure Services** revenue of $81 million, up 39% year-over-year and when adjusted for foreign exchange*',
   'GAAP net income per diluted share of $0.97, up 155% year-over-year and up 156% when adjusted for foreign exchange*, and non-GAAP net income per diluted share* of $1.86, up 17% year-over-year and when adjusted for foreign exchange*',
   'Launched Akamai Inference Cloud, powered by NVIDIA AI infrastructure, to enable AI at the edge',
   'CAMBRIDGE, Mass., Nov. 6, 2025 /PRNewswire/ -- Akamai Technologies, Inc. (NASDAQ: AKAM), the cybersecurity and cloud computing company that powers and pr

In [15]:
scraped_content[0]["content"][22]

,0,1,3,5,7
0,NaN,"Three Months EndingDecember 31, 2025","Three Months EndingDecember 31, 2025","Year Ending December 31, 2025","Year Ending December 31, 2025"
1,NaN,Low End,High End,Low End,High End
2,Revenue (in millions),"$ 1,065","$ 1,085","$ 4,178","$ 4,198"
3,Non-GAAP operating margin *,28 %,30 %,29 %,30 %
4,Non-GAAP net income per diluted share *,$ 1.65,$ 1.85,$ 6.93,$ 7.13
5,Non-GAAP tax rate*,18 %,19 %,19 %,19 %
6,Shares used in non-GAAP per diluted share calc...,147,147,147,147
7,Capex as a percentage of revenue *,16 %,16 %,20 %,20 %
